In [1]:
from data_utils import extract_data, load_data, preprocess_data

In [2]:
data = load_data("data")

In [3]:
X_train, X_test, y_train, y_test = preprocess_data(
    data,
    add_post_text=True,
    include_pooled_comments=False,
    add_bertscore=True,
    n_negative_samples_per_link=1
)

C:\Users\Raman\PycharmProjects\stackexchange-recsys\data_utils.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts_features['body'] = posts_features['body'].map(clean_html)


Calculating BERTScore:   0%|          | 0/160 [00:00<?, ?it/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# for col in X_train.columns:
#     print(X_train[col].)
X_train['body'].iloc[0]

"What's the difference between sautéing, frying, and caramelizing?\nWhen I cook chopped onions in a pan with oil until they are brown and have a slightly sweet taste, which of the above is the correct term for what I am doing?\n"

In [6]:
X_train

,linktypeid,score,viewcount,answercount,commentcount,body,score_related,viewcount_related,answercount_related,commentcount_related,body_related,vote_count,vote_count_related,avg_comment_score,comment_count,avg_comment_score_related,comment_count_related,bertscore_f1
3817,1,10,3828,4,2,"What's the difference between sautéing, frying...",25,7205,3,0,"As I understand it, all of these terms refer t...",10.0,29.0,1.500000,2.0,NaN,NaN,0.864476
2530,0,2,174,1,2,I accidentally added vegetables and water to t...,3,555,2,1,I've recently realized how easy it is to make ...,2.0,3.0,0.000000,2.0,2.000000,1.0,0.842426
2843,0,4,1506,2,2,My mother tells me when she was young her moth...,0,105,1,2,I have light cream with 15% fat. How can I use...,4.0,NaN,0.000000,2.0,0.000000,2.0,0.832239
4421,0,-1,403,2,2,What are the different types of flours used in...,5,799,3,1,Many grains (and some starchy non-grains) can ...,6.0,6.0,0.000000,2.0,2.000000,1.0,0.815287
1547,0,2,2604,3,0,Which properties should I look for or avoid in...,1,153,1,3,I accidentally made way too much rice for a di...,2.0,1.0,NaN,NaN,0.333333,3.0,0.842844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5599,0,2,237,2,1,"The package says cook from frozen, do not thaw...",5,4954,2,0,Most bread recipes give an estimate to how lon...,2.0,6.0,0.000000,1.0,NaN,NaN,0.851326
5776,1,4,155,5,7,I've been given three chilli plants:\n\nBulgar...,1,141,1,3,I don't know how to pickle cayenne and jalapen...,4.0,3.0,0.285714,7.0,0.666667,3.0,0.838088
907,0,3,263,1,0,How do I roll or wrap up pasta into a log-like...,1,1579,1,3,I am trying to make protein bars at home from ...,4.0,1.0,NaN,NaN,0.333333,3.0,0.829248
8136,1,5,63,0,5,\nPossible Duplicate:\nHow long can I store a ...,60,19845,1,0,How do I know if a given food or ingredient I ...,9.0,108.0,0.200000,5.0,NaN,NaN,0.855432


In [7]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6391 entries, 3817 to 7776
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   linktypeid                 6391 non-null   int64  
 1   score                      6391 non-null   object 
 2   viewcount                  6391 non-null   object 
 3   answercount                6391 non-null   object 
 4   commentcount               6391 non-null   object 
 5   body                       6391 non-null   object 
 6   score_related              6391 non-null   object 
 7   viewcount_related          6391 non-null   object 
 8   answercount_related        6391 non-null   object 
 9   commentcount_related       6391 non-null   object 
 10  body_related               6391 non-null   object 
 11  vote_count                 5964 non-null   float64
 12  vote_count_related         6176 non-null   float64
 13  avg_comment_score          4505 non-null   float64

In [8]:
X_train["linktypeid"].value_counts()

linktypeid
0    3435
1    2956
Name: count, dtype: int64

In [9]:
X_test["linktypeid"].value_counts()

linktypeid
0    837
1    761
Name: count, dtype: int64

In [4]:
X_train.drop(columns=["linktypeid"], inplace=True)
X_test.drop(columns=["linktypeid"], inplace=True)

In [5]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score

# Define categorical features indices if you have any
categorical_features_indices = []  # e.g., [0, 1, 2] if your first three columns are categorical

# Initialize CatBoostClassifier
model = CatBoostClassifier(
    iterations=1000,  # Number of boosting iterations
    learning_rate=0.1,  # Learning rate
    depth=6,  # Depth of the tree
    cat_features=categorical_features_indices,  # Specify categorical features
    # text_features=["body", "comments_text", "comments_text_related", "body_related"],
    text_features=["body", "body_related"],
    verbose=100  # Print training progress every 100 iterations
)

# Train the model
model.fit(X_train, y_train, eval_set=(X_test, y_test), plot=True)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6563201	test: 0.6552518	best: 0.6552518 (0)	total: 275ms	remaining: 4m 34s
100:	learn: 0.3223398	test: 0.3675542	best: 0.3675542 (100)	total: 9.82s	remaining: 1m 27s
200:	learn: 0.2301583	test: 0.3445833	best: 0.3445833 (200)	total: 19.4s	remaining: 1m 16s
300:	learn: 0.1784997	test: 0.3364402	best: 0.3361130 (295)	total: 28.8s	remaining: 1m 6s
400:	learn: 0.1414892	test: 0.3361602	best: 0.3348858 (379)	total: 38.5s	remaining: 57.6s
500:	learn: 0.1144138	test: 0.3325977	best: 0.3324247 (498)	total: 48.4s	remaining: 48.2s
600:	learn: 0.0938820	test: 0.3319252	best: 0.3310742 (557)	total: 58s	remaining: 38.5s
700:	learn: 0.0781199	test: 0.3357778	best: 0.3310742 (557)	total: 1m 8s	remaining: 29.1s
800:	learn: 0.0653519	test: 0.3370481	best: 0.3310742 (557)	total: 1m 17s	remaining: 19.3s
900:	learn: 0.0549970	test: 0.3403695	best: 0.3310742 (557)	total: 1m 27s	remaining: 9.61s
999:	learn: 0.0474649	test: 0.3430958	best: 0.3310742 (557)	total: 1m 36s	remaining: 0us

bestTest = 

In [6]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,body_related,29.100615
1,body,28.445814
2,bertscore_recall,8.399147
3,bertscore_precision,7.160863
4,vote_count_related,5.995723
5,bertscore_f1,5.439192
6,score_related,4.090453
7,viewcount_related,3.840413
8,commentcount,1.533732
9,viewcount,1.476563
